In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0'}

In [3]:
base_url = 'https://om.opensooq.com/en/property/property-for-rent'
current_page_url = base_url

In [ ]:
all_apt = {"Title": [], "Location": [], "Price": [], "Size": [], "Listing_Type": []}

In [ ]:
while True:
    print("scarpping page: ", current_page_url)
    try:
        response = requests.get(current_page_url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"error fetching page {current_page_url}: {e}")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    apartment = soup.find_all('div', class_="postDet flex flex-1 flexWrap flexSpaceBetween flexDirectionColumn overflowHidden ripple")
    if not apartment:
        print("no apartment found!")
        break
    for apt in apartment:
        title = apt.find("div", class_="flex flexSpaceBetween gap-10")
        if title:
            title = title.text.strip()
            all_apt["Title"].append(title)
        else:
            all_apt['Title'].append(None)

        location = apt.find("div", class_="flex alignItems gap-5 darkGrayColor")
        if location:
            location = location.text.strip()
            all_apt["Location"].append(location)   
        else:
            all_apt["Location"].append(None) 

        price = apt.find("div", class_="priceColor bold alignSelfCenter font-18 ms-auto")
        if price:
            price = price.text.strip()
            all_apt["Price"].append(price)
        else:
            all_apt["Price"].append(None)

        size = apt.find("div", class_="flex flexSpaceBetween gap-10")
        if size:
            size = size.text.strip()
            all_apt["Size"].append(size)
        else:
            all_apt["Size"].append(None)

        all_apt["Listing_Type"].append("For Rent")
        
    next_button = soup.find('a', class_='blueHoverBg border pagerItem flex alignItems justifyContent radius-8 ripple', attrs={'title':"Go to next page"}, href=True)
    if next_button:
        next_page_relative_url = next_button['href']
        new_url = requests.compat.urljoin(base_url, next_page_relative_url)

    if new_url == current_page_url:
        print("No next page. No more apartment found!")
        break
    else:
        current_page_url = new_url
        time.sleep(5)

scarpping page:  https://om.opensooq.com/en/property/property-for-rent
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=2
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=3
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=4
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=5
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=6
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=7
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=8
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=9
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=10
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=11
scarpping page:  https://om.opensooq.com/en/property/property-for-rent?page=12
scarpping page:  https://om.opensooq.com/en/property/property-for-re

In [8]:
df1 = pd.DataFrame(all_apt)
df1.to_csv("opensooq.csv")